In [1]:
from keys import keys

import json
import pandas as pd
import argparse
import io
import os
from datetime import date

# XE api
from xecd_rates_client import XecdClient

# Google News api
from newsapi import NewsApiClient

# Google Cloud client library api
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types


In [2]:
class Create_DF:
    def __init__(self, cur_original, cur_new, news_api_key, json_google_cloud, xecd_client):
        self.today = date.today().strftime("%Y/%m/%d").replace("/","-")
        self.original = cur_original
        self.new = cur_new
        self.newsapi = NewsApiClient(api_key=news_api_key)
        self.client = language.LanguageServiceClient.from_service_account_json(json_google_cloud)
        self.xecd = xecd_client
        self.approved_classifications = [
            'Finance',
            'Business',
            'Politics',
            'Jobs & Education',
            'Business News',
            'Business & Industrial',
        ]
        
        
    # classify text into categories   
    def __classify(self, text):
        client = self.client
        document = language.types.Document(content=text, type=language.enums.Document.Type.PLAIN_TEXT)
        response = client.classify_text(document)
        categories = response.categories

        result = []

        for category in categories:
            categories = category.name.split("/")
            categories.pop(0)
            for i in range(len(categories)):
                result.append(categories[i])
        return result
    
    def __getRate(self, date, fromCur, toCur):
        xecd = self.xecd
        rateResult = xecd.historic_rate(date, "12:00", fromCur, toCur, 1)
        rateObj={}
        rateObj['date'] = rateResult['timestamp'][:10]
        rateObj['rate'] = rateResult['to'][0]['mid']
        return rateObj['rate']
    
    
    # retrieve articles from google news api
    def __retrieve_articles(self):
        today = self.today
        # grabbing articles
        newsapi = self.newsapi
        
        all_articles = newsapi.get_everything(q=self.original, from_param='2019-08-15', to=today, language='en', sort_by='relevancy')
        articles = all_articles['articles']
        
        formatted_articles = []

        # creating new article object
        for article in articles:
            info = str(article['title']) + " " + str(article['description']) + " " + str(article['content'])
            classifications = self.__classify(info)
            if len(set(classifications).intersection(set(self.approved_classifications))) != 0:
                individual_article = {}
                individual_article['date'] = article['publishedAt'][:10]

                info = info.replace("'","") 
                individual_article['text'] = info

                formatted_articles.append(individual_article)
                
        self.formatted_articles = formatted_articles
#         print(json.dumps(self.formatted_articles, indent=4, sort_keys=True))
        return
    
    
    # Sentiment analysis using google cloud
    def __sentiment_analysis(self):
        client = self.client
        tone_analyzed_articles = {'date':[],'score':[]}
        formatted_articles = self.formatted_articles.copy()

        # The text to analyze
        for i in range(len(formatted_articles)):
            analyzed_article = {}
            article = formatted_articles[i]
            text = article['text']

            document = types.Document(content = text, type = enums.Document.Type.PLAIN_TEXT)

            # Detects the sentiment of the text
            sentiment = client.analyze_sentiment(document=document).document_sentiment
            
            # Appends to dataframe
            tone_analyzed_articles['date'].append(article['date'])
            tone_analyzed_articles['score'].append(sentiment.score)
        
#         print(json.dumps(tone_analyzed_articles, indent=4, sort_keys=True))        
        
        return pd.DataFrame(tone_analyzed_articles)
        
    # Dump the dataframe
    def __dump_df(self, df):
        df.to_csv(r'data/articles/' + self.original + '.csv')
        
        
    # Create the dataframe and save it
    def df_create(self):
        self.__retrieve_articles()
        df = self.__sentiment_analysis()
        self.__dump_df(df)
        return df

## Input Parameter

In [3]:
news_api_key = keys['news']
json_google_cloud = keys['json_google_cloud']
xecd_client = keys['xe']

In [4]:
print("Enter the original currency: ")
# cur_original = input()
print("Enter the new currency: ")
# cur_return = input()

Enter the original currency: 
Enter the new currency: 


In [5]:
cur_original = 'CAD'
cur_return = 'USD'

## Create Dataframe

In [6]:
created_df = Create_DF(cur_original, cur_return, news_api_key, json_google_cloud, xecd_client)
df = created_df.df_create()
df.head()

,date,score
0,2019-08-20,-0.4
1,2019-08-23,0.8
2,2019-08-29,0.0
3,2019-08-22,0.0
4,2019-09-13,0.0
